# Acceleration Prediction using LeNet-inspired Architecture

## Purpose
To predict the acceleration of the vehicle based on the image frames in the video. 

We use the **3-layered Convnet Architecture design inspired by LeNet, 1998 paper by Le Cunn.**


## Custom-Defined Functions

In [2]:
import scipy.misc
import random
import numpy as np

xs = []
ys = []
accels = []
brake = []
# gear = []
opticalFlow = []

#points to the end of the last batch
train_batch_pointer = 0
val_batch_pointer = 0
corr_train_batch_pointer = 0
corr_val_batch_pointer = 0

dataPath = "indian_dataset/"
corrDataPath = "indian_dataset/corr/"
fileNamePrefix = "circuit2_x264.mp4 "


with open(dataPath+"data.txt") as f:
    for line in f:
        xs.append(scipy.misc.imresize(scipy.misc.imread(
            dataPath + fileNamePrefix + str(int(line.split()[0])).zfill(5)+".jpg")[-150:], [112, 112]) / 255.0)
        #the paper by Nvidia uses the inverse of the turning radius,
        #but steering wheel angle is proportional to the inverse of turning radius
        #so the steering wheel angle in radians is used as the output
        accels.append(float(line.split()[2])* scipy.pi / 180)


#get number of images
num_images = len(xs)

# train_xs, train_accels, val_xs, val_accels
train_xs = xs[:int(len(xs) * 0.8)]
train_accels = accels[:int(len(xs) * 0.8)]

val_xs = xs[-int(len(xs) * 0.2):]
val_accels = accels[-int(len(xs) * 0.2):]

# print(train_accels)
# print(len(train_xs))

# train_xs, train_accels, val_xs, val_accels
train_xs = np.array(train_xs)
train_accels = np.array(train_accels)

val_xs = np.array(val_xs)
val_accels = np.array(val_accels)

input_shape = (112, 112, 3)

c:\python\lib\site-packages\ipykernel_launcher.py:26: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
c:\python\lib\site-packages\ipykernel_launcher.py:26: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


## Model 1: LeNet Inspired 3-Convolution Layer Architecture

This 3-layered is different but inspired from the LeNet, 1998 paper by Le Cunn.

http://yann.lecun.com/exdb/publis/pdf/lecun-01a.pdf


<img src="lenet.jpg" width="700">

In [2]:
# The model is inspired from the LeNet, 1998 paper by Le Cunn

# Credits: https://github.com/keras-team/keras/blob/master/examples/mnist_cnn.py

from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

from keras.layers.normalization import BatchNormalization
# import seaborn as sns

batch_size = 128
epochs = 30

model = Sequential()
model.add(Conv2D(256, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape)) #Convolution
model.add(MaxPooling2D(pool_size=(2, 2)))  #Subsampling
model.add(Dropout(0.25))
# model.add(BatchNormalization())

model.add(Conv2D(128, (3, 3), activation='relu')) #Convolution
model.add(MaxPooling2D(pool_size=(2, 2)))         #Subsampling
model.add(Dropout(0.25))
# model.add(BatchNormalization())

# model.add(Conv2D(256, (3, 3), activation='relu'))
# model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(128, activation='relu')) # Full Connection
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu')) # Full Connection
model.add(Dropout(0.5))
model.add(Dense(1))

model.compile(loss='mse',#loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['mean_squared_error', 'mean_absolute_error', 'mean_absolute_percentage_error', 'cosine_proximity'])


# train_xs, train_accels, val_xs, val_accels
history=model.fit(train_xs, train_accels,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(val_xs, val_accels))

score = model.evaluate(val_xs, val_accels, verbose=0)
# print('Test loss:', score[0])
# print('Test accuracy:', score[1])

# plotGraph(history=history)
# plotWeightM1(model=model)



Using TensorFlow backend.


Train on 23760 samples, validate on 5940 samples
Epoch 1/30
23760/23760 [==============================] - 72s 3ms/step - loss: 0.0027 - mean_squared_error: 0.0027 - mean_absolute_error: 0.0341 - mean_absolute_percentage_error: 665310.8763 - cosine_proximity: -0.9530 - val_loss: 0.0012 - val_mean_squared_error: 0.0012 - val_mean_absolute_error: 0.0313 - val_mean_absolute_percentage_error: 1629.3925 - val_cosine_proximity: -1.0000
Epoch 2/30
23760/23760 [==============================] - 67s 3ms/step - loss: 9.8084e-04 - mean_squared_error: 9.8084e-04 - mean_absolute_error: 0.0266 - mean_absolute_percentage_error: 544773.0175 - cosine_proximity: -0.9415 - val_loss: 0.0012 - val_mean_squared_error: 0.0012 - val_mean_absolute_error: 0.0303 - val_mean_absolute_percentage_error: 1744.3755 - val_cosine_proximity: -1.0000
Epoch 3/30
23760/23760 [==============================] - 66s 3ms/step - loss: 8.6685e-04 - mean_squared_error: 8.6685e-04 - mean_absolute_error: 0.0248 - mean_absolute_perc

Epoch 22/30
23760/23760 [==============================] - 66s 3ms/step - loss: 3.4575e-04 - mean_squared_error: 3.4575e-04 - mean_absolute_error: 0.0149 - mean_absolute_percentage_error: 223506.8909 - cosine_proximity: -0.8793 - val_loss: 0.0012 - val_mean_squared_error: 0.0012 - val_mean_absolute_error: 0.0294 - val_mean_absolute_percentage_error: 1695.1045 - val_cosine_proximity: -1.0000
Epoch 23/30
23760/23760 [==============================] - 66s 3ms/step - loss: 3.3386e-04 - mean_squared_error: 3.3386e-04 - mean_absolute_error: 0.0146 - mean_absolute_percentage_error: 205750.2334 - cosine_proximity: -0.8787 - val_loss: 0.0012 - val_mean_squared_error: 0.0012 - val_mean_absolute_error: 0.0284 - val_mean_absolute_percentage_error: 1765.2763 - val_cosine_proximity: -1.0000
Epoch 24/30
23760/23760 [==============================] - 66s 3ms/step - loss: 3.2564e-04 - mean_squared_error: 3.2564e-04 - mean_absolute_error: 0.0144 - mean_absolute_percentage_error: 229049.8341 - cosine_pro

In [16]:
# To try predict acceleration using train data
pred = model.predict(train_xs)

#Print acceleration first n frames
for i in pred[0:500]:
    print(i*180/scipy.pi)

[0.92357355]
[0.92357355]
[0.9310391]
[0.9301475]
[0.9338954]
[0.9341978]
[0.94233406]
[0.93697715]
[0.9277656]
[0.931919]
[0.9339071]
[0.9301183]
[0.9293895]
[0.9270301]
[0.9313618]
[0.94097805]
[0.9448145]
[0.9453828]
[0.9402479]
[0.94632584]
[0.94994295]
[0.9340492]
[0.9422594]
[0.93306893]
[0.94362503]
[0.9397941]
[0.9537025]
[0.94687253]
[0.9523851]
[0.9354172]
[0.9383295]
[0.9428991]
[0.94260913]
[0.94029]
[0.9449467]
[0.9489774]
[0.942226]
[0.93854314]
[0.9425991]
[0.95988804]
[0.94689965]
[0.948667]
[0.9394076]
[0.9394108]
[0.94459707]
[0.93810683]
[0.9388357]
[0.9477207]
[0.9370355]
[0.9421201]
[0.9484173]
[0.94396645]
[0.948088]
[0.9414139]
[0.9583285]
[0.9663747]
[0.95048684]
[0.95292795]
[0.9477265]
[0.9556924]
[0.9527644]
[0.9528844]
[0.95880514]
[0.9665853]
[0.9427091]
[0.94912565]
[0.9515617]
[0.96681327]
[0.96333563]
[0.9631466]
[0.95660615]
[0.95560145]
[0.96558595]
[0.97763824]
[0.97463447]
[0.98999953]
[0.98700166]
[0.98382944]
[0.9916441]
[0.9790558]
[0.9685356]
[0.

In [4]:
# model.save("save/model_accel.ckpt")